In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 29783050
paper_name = 'jiang_papadopoulos_2018' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_csv('raw_data/hits.txt', sep='\t', header=None, names=['orfs'])

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 96 x 1


In [7]:
original_data.head()

,orfs
0,YAL013W
1,YAL023C
2,YBR036C
3,YBR126C
4,YCR034W


In [8]:
original_data['orfs'] = original_data['orfs'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orfs'] = clean_orf(original_data['orfs'])

In [10]:
# Translate to ORFs 
original_data['orfs'] = translate_sc(original_data['orfs'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orfs'])

In [12]:
print(original_data.loc[~t,])

Empty DataFrame
Columns: [orfs]
Index: []


In [13]:
original_data['score'] = -1

In [14]:
# If the same strain is present more than once, average its values
data = original_data.groupby('orfs')['score'].mean().to_frame()

In [16]:
data.index.name='orf'

In [15]:
print('Final data dimensions: %d x %d' % (data.shape))

Final data dimensions: 96 x 1


# Prepare the final dataset

In [18]:
dataset_ids = [16444]
datasets = datasets.reindex(index=dataset_ids)

In [19]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [20]:
data.head()

dataset_id,16444
data_type,value
orf,
YAL013W,-1
YAL023C,-1
YBL093C,-1
YBR036C,-1
YBR126C,-1


## Subset to the genes currently in SGD

In [21]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 1


In [22]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

In [23]:
data.head()

,dataset_id,16444
,data_type,value
gene_id,orf,
11,YAL013W,-1
21,YAL023C,-1
181,YBL093C,-1
232,YBR036C,-1
320,YBR126C,-1


# Normalize

In [24]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [25]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [26]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

In [27]:
data_all.head()

dataset_id      16444          
data_type       value    valuez
gene_id orf                    
11      YAL013W    -1 -6.929722
21      YAL023C    -1 -6.929722
181     YBL093C    -1 -6.929722
232     YBR036C    -1 -6.929722
320     YBR126C    -1 -6.929722

# Print out

In [28]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [29]:
from IO.save_data_to_db3 import *

In [30]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 29783050...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  3.36it/s]

Updating the data_modified_on field...
